# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

### Imports

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Hyperparameters

In [ ]:
# TODO: Set hyperparameters.
training_batch_size = 64
validation_batch_size = 64
test_batch_size = 64
epochs = 0
learning_rate = 0

### Device

In [ ]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

### Load Dataset

In [ ]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=[transforms.Resize(256), transforms.ToTensor()],
    download=True
)

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=[transforms.Resize(256), transforms.ToTensor()],
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=[transforms.Resize(256), transforms.ToTensor()],
    download=True
)

### DataLoaders

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)   

## Model

In [ ]:
onehot = F.one_hot(torch.tensor([e for e in range(0,102)]), num_classes=102)
onehot

In [ ]:
# Each image is 591x500 pixels

class NeuralNetwork(nn.Module):
	def __init__(self):
		super(NeuralNetwork, self).__init__()
		self.layers = nn.Sequential(
			nn.ReLU(),
   			nn.Conv2d(1, 6, (5,5), (1,1), (0,0)),
      		nn.ReLU(),
        	nn.Conv2d(6, 16, (5,5), (1,1), (0,0)),
         	nn.ReLU(),
         	nn.Conv2d(16, 120, (5,5), (1,1), (0,0)),
			nn.Linear(120, 102)
   		)

	def forward(self, x):
		x = self.layers(x)
		return x
  

In [ ]:
mat = scipy.io.loadmat('data/flowers-102/imagelabels.mat')
print(set(mat['labels'][0]))

# The labels go from 1-102 therefore the output needs to be 1-102